In [0]:
!pip install numpy scipy gensim nltk

import warnings
warnings.filterwarnings('ignore')

import numpy as np

In [0]:
import gensim
from gensim.models import word2vec
! wget http://mattmahoney.net/dc/text8.zip -P /tmp
! unzip /tmp/text8.zip -d /tmp/
! head -c10000000 /tmp/text8 > /tmp/text8_10mo
 



In [0]:
# lire les donnés que nous avons téléchargé
sentences = word2vec.Text8Corpus('/tmp/text8_10mo')
for s in sentences:
  print("a sentence",s[0:20])
  break

In [0]:
# Apprendre un modèle de embeddings sur text8
params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 5,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

my_model = word2vec.Word2Vec(sentences, **params)
print(my_model)


In [0]:
word = 'services'
most_similar = my_model.wv.most_similar(positive=[word], topn=5)
print('word',word,'most_similar: ', most_similar)

word = 'japanese'
most_similar = my_model.wv.most_similar(positive=[word], topn=5)
print('word',word,'most_similar: ', most_similar)

word = 'six'
most_similar = my_model.wv.most_similar(positive=[word], topn=5)
print('word',word,'most_similar: ', most_similar)

word = 'good'
most_similar = my_model.wv.most_similar(positive=[word], topn=5)
print('word',word,'most_similar: ', most_similar)


# Calcul de la similarité cosinus entre deux mots
# cos_sim( a,b) =  a*b/|a||b|
cos_sim = np.dot( my_model.wv['services'],my_model.wv['facilities'] )/(np.linalg.norm(my_model.wv['services'], ord=2)*np.linalg.norm(my_model.wv['facilities'], ord=2))
print('consine similarity between', 'services' , 'facilities', cos_sim)


In [0]:
# Relations semantiques dans les embeddings
most_similar = my_model.wv.most_similar(positive=['woman', 'husband'], negative=['man'], topn=5)
print('most_similar: ', most_similar)


### How to select your parameters?

Les deux parametres les plus importants sont la taille de la fenêtre et le nombre minimal de occurrences

La taille de fenêtre a un impact direct sur ce qui est modelisé :

- Des petites fenetres donnent des embeddings très syntaxiques (similarity)

- Des fenetres plus grandes donnent des embeddings plus sémantiques (relatedness)


In [0]:
# Importance de la taille de fenetre
params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 15,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

largecontext_model = word2vec.Word2Vec(sentences, **params)


params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 2,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

smallcontext_model = word2vec.Word2Vec(sentences, **params)





In [0]:
word = 'food'
most_similar = largecontext_model.wv.most_similar(positive=[word], topn=5)
print('Using a large context... word',word,'most_similar: ', most_similar)

most_similar = smallcontext_model.wv.most_similar(positive=[word], topn=5)
print('Using a small context... word',word,'most_similar: ', most_similar)




word = 'english'
most_similar = largecontext_model.wv.most_similar(positive=[word], topn=5)
print('Using a large context... word',word,'most_similar: ', most_similar)

most_similar = smallcontext_model.wv.most_similar(positive=[word], topn=5)
print('Using a small context... word',word,'most_similar: ', most_similar)



### Stabilité des Embeddings

L'information utile dans les embeddings est encodé dans les similarités entre les mots et non pas dans leur position dans l'espace

Plusieurs executions de W2V sur les mêmes données produissent des embeddings très similaires.

C'est-à-dire, les plus proche voisins de chaque mot ne changent presque pas.

Mais malheureusement, deux modèles d'embeddings ne partagent pas le même espace de representation

In [0]:
# Nous allons apprendre deux embeddings avec les même parametres
params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 5,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

model_run1 = word2vec.Word2Vec(sentences, **params)
model_run2 = word2vec.Word2Vec(sentences, **params)


In [0]:
# Nous devrions trouver a peu près les mêmes voisins pour tous les mots
word = 'france'
most_similar_run1 = model_run1.wv.most_similar(positive=[word], topn=10)
most_similar_run2 = model_run2.wv.most_similar(positive=[word], topn=10)
for r1,r2 in zip(most_similar_run1,most_similar_run2):
  print('run1',r1,'run2',r2)


In [0]:
# Les similarités sont les mêmes, mais malheuresement les coordonées sont très differents 

word = 'france'
print(model_run1.wv[word])
print(model_run2.wv[word])

# We can compute the cosine similarity across models
# If the models were compatible the similarity ~1 for every word.
cos_sim = np.dot( model_run1.wv[word],model_run2.wv[word] )/(np.linalg.norm(model_run1.wv[word], ord=2)*np.linalg.norm(model_run2.wv[word], ord=2))
print('consine similarity across models for word:', word , 'is', cos_sim)


### Alignement des embeddings après apprentissage

Voici comment aligner des embeddigs dans un même espace.

Il y a plusieurs methodes. La plus simple consiste a utiliser un dictionnaire de mots de reference a aligner.

Pour construire une la transformation lineaire qui nous envoie d'un espace vers l'autre


In [0]:
# try different values of the number of pivots. The more words used during the alignment the better
nb_pivots = 500
wordlist = list(model_run1.wv.vocab.keys())[0:nb_pivots]
model_run1.init_sims()
model_run2.init_sims()


# Extraire le vocabulaire de chaque model
vocab_m1 = set(model_run1.wv.vocab.keys())
vocab_m2 = set(model_run2.wv.vocab.keys())

# Extraire le vocabulaire en commun avec la liste 
common_vocab = list(vocab_m1&vocab_m2&set(wordlist))

syn0norms = []
# pour chaque modèle...
for m in [model_run1,model_run2]:
	# Remplacer syn0norm avec un nouveau array uniquement avec le mots en commun
	indices = [m.wv.vocab[w].index for w in common_vocab]
	old_arr = m.wv.syn0norm
	new_arr = np.array([old_arr[index] for index in indices])
	syn0norms.append( new_arr )
  
base_vecs = syn0norms[0]
other_vecs = syn0norms[1]

# produit scalaire entre les vecteurs de chaque espace
m = other_vecs.T.dot(base_vecs) 

# SVD method 
u, _, v = np.linalg.svd(m)
ortho = u.dot(v) 

# Replacer les embeddings avec les projections
model_run2.wv.syn0norm = model_run2.wv.syn0 = (model_run2.wv.syn0norm).dot(ortho)


word = 'france'
print(model_run1.wv[word])
print(model_run2.wv[word])

# We can compute the cosine similarity across models
# If the models were compatible the similarity ~1 for every word.
cos_sim = np.dot( model_run1.wv[word],model_run2.wv[word] )/(np.linalg.norm(model_run1.wv[word], ord=2)*np.linalg.norm(model_run2.wv[word], ord=2))
print('consine similarity across models for word:', word , 'is', cos_sim)




### Mots hors vocabulaire

Je peux apprendre un embedding pour les mots inconnues avec word2vec. 

Par exemple, je peux reemplace tous les mots rares (count<5) par un token "_oov_"  et apprendre mes embeddings normalement

In [0]:
from collections import Counter

count = Counter([x for sent in sentences for x in sent])
oov_sentences = []
for sent in sentences:
  oov_sentences.append( [ s if(count[s]>5) else '_oov_' for s in sent ] )

  
for oov_sent,sent in zip(list(oov_sentences)[0:5], list(sentences)[0:5]):
  print(oov_sent[0:20])
  print(sent[0:20])
  

params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 15,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

oov_model = word2vec.Word2Vec(oov_sentences, **params)
print( 'oov_embedding', oov_model.wv['_oov_'] )